##### Команда игры «Мир анализа данных» ищет способы привлечь новую аудиторию. Одним из них является публикация статьи о развитии индустрии игр в начале XXI века, плавно подводящая к игре «Мир анализа данных». В статье-исследовании необходим обзор игровых платформ, изучение объёмов продаж игр разных жанров и региональные предпочтения игроков — акцент на играх жанра RPG, изучение развития игровой индустрии с 2000 по 2013 год на основе исторических данных, собранных из открытых источников. Эти данные содержат информацию о продажах игр, сделанных в разных жанрах и выпущенных на разных платформах, а также пользовательские и экспертные оценки игр

##### Структура работы:
##### 1. Загрузка и знакомство с данными
##### 2. Проверка ошибок в данных и их предобработка:
    2.1 Названия или метки столбцов датафрейма
    2.2 Наличие пропусков в данных
    2.3 Типы данных
    2.4 Явные и неявные дубликаты в данных
        
##### 3. Фильтрация данных
##### 4. Категоризация данных:
    4.1 Оценки пользователей
    4.2 Оценки критиков
    4.3 Топ-7 платформ
---

##### 1. Загрузка и знакомство с данными

In [1]:
import pandas as pd
import re

In [2]:
url = 'new_games.csv'

In [3]:
df = pd.read_csv(url)

In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  str    
 1   Platform         16956 non-null  str    
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  str    
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  str    
 6   JP sales         16956 non-null  str    
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  str    
 10  Rating           10085 non-null  str    
dtypes: float64(4), str(7)
memory usage: 1.4 MB


In [5]:
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Были загружены данные датасета `game_trends.csv` Они содержат 11 столбцов и 16956 строк, в которых представлена информация о о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр. При первичном знакомстве с данными и их предобработкой получены следующие результаты:
- по шести столбцам (`Name`, `Year of Release`, `Genre`, `Critic Score`, `User Score`, `Rating`) были обнаружены пропущенные значения;
- типы данных некоторых столбцов, необходимо оптимизировать (например `Year of Release`);
- необходимо также привести названия к единому стилю snake case;
- выполнить предобработку данных в части пропусков

##### 2. Проверка ошибок в данных и их предобработка

Для самопроверки и отслеживания изменений создадим переменные, считающее количество строк и пропусков - до начала предобработки


In [6]:
initial_rows = df.shape[0]
initial_na_count = df.isna().sum()

    2.1 Названия или метки столбцов датафрейма:

Выведем названия столбцов списком:

In [7]:
list(df.columns)

['Name',
 'Platform',
 'Year of Release',
 'Genre',
 'NA sales',
 'EU sales',
 'JP sales',
 'Other sales',
 'Critic Score',
 'User Score',
 'Rating']

Приведем их названия к стилю snake case, применим функцию с регулярными выражениями к столбцам датафрейма

In [8]:
def to_snake_case(text):
    text = re.sub(r'[\s-]+', '_', text)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', text)
    return text.lower()

In [9]:
df.columns = [to_snake_case(col) for col in df.columns]

Выполним проверку изменений:

In [10]:
list(df.columns)

['name',
 'platform',
 'year_of_release',
 'genre',
 'na_sales',
 'eu_sales',
 'jp_sales',
 'other_sales',
 'critic_score',
 'user_score',
 'rating']

    2.2 Наличие пропусков в данных:

In [11]:
missing_absolute = df.isna().sum()
missing_relative = (df.isna().mean() * 100)
missing_absolute = missing_absolute[missing_absolute > 0]
missing_relative = missing_relative[missing_relative > 0].sort_values(ascending=False).round(2)
if not missing_absolute.empty:
    print(f"Пропущенных строк в столбцах датафрейма в абсолютных значениях\n{missing_absolute}")
    print("\n")
    print(f"Пропущенных строк в столбцах датафрейма в относительных значениях\n{missing_relative}")
else:
    print("Пропусков в данных нет")

Пропущенных строк в столбцах датафрейма в абсолютных значениях
name                  2
year_of_release     275
genre                 2
critic_score       8714
user_score         6804
rating             6871
dtype: int64


Пропущенных строк в столбцах датафрейма в относительных значениях
critic_score       51.39
rating             40.52
user_score         40.13
year_of_release     1.62
name                0.01
genre               0.01
dtype: float64


Столбец `critic_score` (оценка критиков) больше чем наполовину оказался не заполнен. Предположим, это могло произойти по вполне объяснимой причине отсутствия рецензий на игру критиков от изданий (небольшие проекты, недостаточно внимания к проекту). Вариант объясняющий это недавним релизом игры не рассматриваем, ведь разброс отсутствия оценок неравномерно распределен по году их выпуска. Возможно также из-за того что данные собраны по задумке проекта коллегами вручную из открытых источников, их просто не было. В данном случае считаю что лучшим вариантом, заполнить пропуски `NaN` значениями в зависимости от аналогичных совпадений столбцов жанра игры `genre`, платформы на которой она была выпущена `platform` и года выпуска `year_of_release`, дабы не потерять важные данные при аргументации в статье

Третий по показателю пропусков столбец `user_score` (оценка пользователей) - по полученным данным отсутствует около 40% значений! Эти показатели не менее важны, поэтому пропуски `NaN` аналогично будут заполнены в зависимости от совпадений по столбцам жанра игры `genre`, платформы на которой она была выпущена `platform` и года выпуска `year_of_release`

Показатели столбца `rating` (рейтинг организации ESRB) нуждаются в дополнительном пояснении. В контексте изучения коллегами исторических данных игровой индустрии, данные этого столбца будут вспомогательным и как мне кажется необязательным элементом при анализе игр. Однако, для полноты исследования сохраним их. К тому же терять 40% в конкретном случае нецелесообразно. Заполнить пропуски `NaN` нулевыми значениями будет неправильно, необходимо присвоить категорию ESRB - RP, Rating Pending (используется в рекламных материалах для игр, которым ещё не присвоен окончательный рейтинг ESRB) или соответствующую, но с учетом аналогичных данных для других столбцов по жанру игры `genre`, платформы на которой она была выпущена `platform` и года выпуска `year_of_release`

С показателями пропусков столбцов `year_of_release`, `name`, `genre` все более прозаично - избавимся от них. Они составляют около 1% от всех данных и не повлияют на результат исследования

In [12]:
df.dropna(subset=['year_of_release', 'name', 'genre'], inplace=True)

In [13]:
missing_absolute_afterdropna = df.isna().sum()
missing_relative_afterdropna = (df.isna().mean() * 100)
missing_absolute_afterdropna = missing_absolute_afterdropna[missing_absolute_afterdropna > 0]
missing_relative_afterdropna = missing_relative_afterdropna[missing_relative_afterdropna > 0].sort_values(ascending=False).round(2)
if not missing_absolute_afterdropna.empty:
    print(f"Пропущенных строк в столбцах датафрейма в абсолютных значениях\n{missing_absolute_afterdropna}")
    print("\n")
    print(f"Пропущенных строк в столбцах датафрейма в относительных значениях\n{missing_relative_afterdropna}")
else:
    print("Пропусков в данных нет")

Пропущенных строк в столбцах датафрейма в абсолютных значениях
critic_score    8594
user_score      6705
rating          6778
dtype: int64


Пропущенных строк в столбцах датафрейма в относительных значениях
critic_score    51.53
rating          40.64
user_score      40.20
dtype: float64


Заполним пропуски для `rating`

In [14]:
def fill_missing_ratings(df):
    # функция для заполнения пропусков
    def fill_rating(row):
        if pd.isna(row['rating']):
            # ищем другие значения в той же группе
            group = df[
                (df['genre'] == row['genre']) &
                (df['platform'] == row['platform']) &
                (df['year_of_release'] == row['year_of_release'])
            ]
            
            # если есть другие значения в группе
            if not group['rating'].isna().all():
                return group['rating'].dropna().iloc[0]
            else:
                return 'RP'
        return row['rating']
    
    # применяем функцию к каждой строке
    df['rating'] = df.apply(fill_rating, axis=1)
    
    return df

df = fill_missing_ratings(df)

Заполним пропуски для `user_score`, но учтем что в уникальных значениях столбца, содержатся строки 'tbd' (предположим - to be determined), их тоже необходимо заполнить

In [15]:
def fill_missing_scores(df):
    # преобразуем значения в числовой формат и заменяем 'tbd' на NaN
    df['user_score'] = pd.to_numeric(df['user_score'].replace('tbd', None), errors='coerce')
    
    # сохраняем групповые средние
    group_means = df.groupby(['genre', 'platform', 'year_of_release'])['user_score'].transform('mean')
    
    # сохраняем словарь групповых средних
    group_means_dict = df.groupby(['genre', 'platform', 'year_of_release'])['user_score'].mean().to_dict()
    
    # заполняем пропуски групповыми средними
    df['user_score'] = df['user_score'].fillna(group_means)
    
    # сохраняем глобальное среднее
    global_mean = df['user_score'].mean()
    
    # заполняем оставшиеся пропуски глобальным средним
    df['user_score'] = df['user_score'].fillna(global_mean)
    
    return df

df = fill_missing_scores(df)

Заполним пропуски для `critic_score`

In [16]:
def fill_missing_scores(df):
    # преобразуем значения в числовой формат
    df['critic_score'] = pd.to_numeric(df['critic_score'], errors='coerce')
    
    # сохраняем групповые средние
    group_means = df.groupby(['genre', 'platform', 'year_of_release'])['critic_score'].transform('mean')
    
    # заполняем пропуски групповыми средними
    df['critic_score'] = df['critic_score'].fillna(group_means)
    
    # сохраняем глобальное среднее
    global_mean = df['critic_score'].mean()
    
    # заполняем оставшиеся пропуски глобальным средним
    df['critic_score'] = df['critic_score'].fillna(global_mean)
    
    return df

df = fill_missing_scores(df)

In [17]:
missing_absolute_afterfilling = df.isna().sum()
missing_relative_afterfilling = (df.isna().mean() * 100)
missing_absolute_afterfilling = missing_absolute_afterfilling[missing_absolute_afterfilling > 0]
missing_relative_afterfilling = missing_relative_afterfilling[missing_relative_afterfilling > 0].sort_values(ascending=False).round(2)
if not missing_absolute_afterfilling.empty:
    print(f"Пропущенных строк в столбцах датафрейма в абсолютных значениях\n{missing_absolute_afterfilling}")
    print("\n")
    print(f"Пропущенных строк в столбцах датафрейма в относительных значениях\n{missing_relative_afterfilling}")
else:
    print("Пропусков в данных нет")

Пропусков в данных нет


    2.3 Типы данных:

Проверим встречаются ли некорректные типы данных и проведем их преобразование в случае необходимости:

In [18]:
df.info()

<class 'pandas.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  str    
 1   platform         16679 non-null  str    
 2   year_of_release  16679 non-null  float64
 3   genre            16679 non-null  str    
 4   na_sales         16679 non-null  float64
 5   eu_sales         16679 non-null  str    
 6   jp_sales         16679 non-null  str    
 7   other_sales      16679 non-null  float64
 8   critic_score     16679 non-null  float64
 9   user_score       16679 non-null  float64
 10  rating           16679 non-null  str    
dtypes: float64(5), str(6)
memory usage: 1.5 MB


Оптимизируем типы данных столбцов датафрейма с понижением разрядности (необходимо понимать, что представленные типы данных для числовых значений стоит рассматривать как ожидаемый результат после автоподбора):

- `platform` - object: преобразуем в категорию, так как это ограниченный набор значений;
- `year_of_release` - int64: оптимизируем до int16, так как диапазон лет ограничен;
- `genre` - object: аналогично платформе преобразуем в категорию, ведь это ограниченный набор значений;
- `na_sales` - float64: попробуем понизить до float32;
- `eu_sales` - object: очевидно должен быть числовым, попробуем преобразовать в float32;
- `jp_sales` - object: также попробуем преобразовать в float32;
- `other_sales` - float64: попробуем понизить до float32;
- `critic_score` - float64: преобразуем в int8, так как оценки критиков целые числа и варьируются от 0 до 100;
- `user_score` - object: преобразуем в float16;
- `rating` - object: преобразуем в категорию, так как рейтинг  в данном случае строго ограничен

In [19]:
# преобразуем названия в строковый тип
df['name'] = df['name'].astype('string')

# преобразуем категориальные переменных
df['platform'] = df['platform'].astype('category')
df['genre'] = df['genre'].astype('category')
df['rating'] = df['rating'].astype('category')

# преобразуем напрямую, так как точно знаем что там только год
df['year_of_release'] = df['year_of_release'].astype('int16')

# преобразуем напрямую оценки критиков, так как диапазон известен
df['critic_score'] = df['critic_score'].astype('int8')

# преобразуем числовые типы с применением автоматического выбора пониженного разряда
df['other_sales'] = pd.to_numeric(df['other_sales'], downcast='float')
df['na_sales'] = pd.to_numeric(df['na_sales'], downcast='float')
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce', downcast='float')
df['jp_sales'] = pd.to_numeric(df['jp_sales'], errors='coerce', downcast='float')
df['user_score'] = pd.to_numeric(df['user_score'], downcast='float')

В ходе преобразования числовых типов столбцов `eu_sales` и `jp_sales` происходит ошибка, это говорит о наличии не числовых данных в них, но это и не `NaN` - так как мы уже проставили их ранее, поэтому игнорируя ошибку и проверяем: преобразование прошло успешно, значит заменим проблемные строки полей медианными значениями и проверим количество строк выведя информацию о датафрейме


In [20]:
df['eu_sales'] = df['eu_sales'].fillna(df['eu_sales'].median())
df['jp_sales'] = df['jp_sales'].fillna(df['jp_sales'].median())

In [21]:
df.info()

<class 'pandas.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16679 non-null  string  
 1   platform         16679 non-null  category
 2   year_of_release  16679 non-null  int16   
 3   genre            16679 non-null  category
 4   na_sales         16679 non-null  float32 
 5   eu_sales         16679 non-null  float32 
 6   jp_sales         16679 non-null  float32 
 7   other_sales      16679 non-null  float32 
 8   critic_score     16679 non-null  int8    
 9   user_score       16679 non-null  float32 
 10  rating           16679 non-null  category
dtypes: category(3), float32(5), int16(1), int8(1), string(1)
memory usage: 684.6 KB


    2.4 Явные и неявные дубликаты в данных:

Нормализуем текстовые данные, приведя их к единому регистру:

In [22]:
# приводим названия игр к нижнему регистру
df['name'] = df['name'].str.lower()

# приводим жанры к нижнему регистру
df['genre'] = df['genre'].str.lower()

# приводим платформы к верхнему регистру
df['platform'] = df['platform'].str.upper()

# приводим рейтинги к верхнему регистру
df['rating'] = df['rating'].str.upper()

print("Проверка формата данных")
print(f"Название\n{df['name'].head(3).tolist()}")
print(f"Жанр\n{df['genre'].head(3).tolist()}")
print(f"Платформа\n{df['platform'].head(3).tolist()}")
print(f"Рейтинг\n{df['rating'].head(3).tolist()}")

Проверка формата данных
Название
['wii sports', 'super mario bros.', 'mario kart wii']
Жанр
['sports', 'platform', 'racing']
Платформа
['WII', 'NES', 'WII']
Рейтинг
['E', 'RP', 'E']


In [23]:
df.info()

<class 'pandas.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  string 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  int16  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float32
 5   eu_sales         16679 non-null  float32
 6   jp_sales         16679 non-null  float32
 7   other_sales      16679 non-null  float32
 8   critic_score     16679 non-null  int8   
 9   user_score       16679 non-null  float32
 10  rating           16679 non-null  object 
dtypes: float32(5), int16(1), int8(1), object(3), string(1)
memory usage: 1.0+ MB


А теперь выведем статистику произведенных изменений (на основе переменных, заданных до начала работы с данными):

In [24]:
final_rows = df.shape[0]
rows_removed = initial_rows - final_rows
percent_removed = (rows_removed / initial_rows) * 100

print(f'В процессе обработки данных было удалено {rows_removed} строк, что составляет примерно {percent_removed:.2f}% от общего объема данных')
print(f'- Все столбцы после обработки имеют одинаковое количество значений ({final_rows})')
print(f'- Общее количество строк уменьшилось на {rows_removed}')

В процессе обработки данных было удалено 277 строк, что составляет примерно 1.63% от общего объема данных
- Все столбцы после обработки имеют одинаковое количество значений (16679)
- Общее количество строк уменьшилось на 277


##### 3. Фильтрация данных

Для изучения истории продаж игр в начале XXI века, отфильтруем данные по кретерию период с 2000 по 2013 год включительно. Новый срез данных сохраним в датафрейме - `df_actual`:


In [25]:
# датафрейм с данными за период 2000-2013
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]

print("\nРаспределение игр по годам:")
print(df_actual['year_of_release'].value_counts().sort_index())


Распределение игр по годам:
year_of_release
2000     357
2001     491
2002     839
2003     789
2004     771
2005     950
2006    1020
2007    1218
2008    1445
2009    1450
2010    1279
2011    1149
2012     670
2013     552
Name: count, dtype: int64


##### 4. Категоризация данных

In [26]:
df_actual.info()

<class 'pandas.DataFrame'>
Index: 12980 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12980 non-null  string 
 1   platform         12980 non-null  object 
 2   year_of_release  12980 non-null  int16  
 3   genre            12980 non-null  object 
 4   na_sales         12980 non-null  float32
 5   eu_sales         12980 non-null  float32
 6   jp_sales         12980 non-null  float32
 7   other_sales      12980 non-null  float32
 8   critic_score     12980 non-null  int8   
 9   user_score       12980 non-null  float32
 10  rating           12980 non-null  object 
dtypes: float32(5), int16(1), int8(1), object(3), string(1)
memory usage: 798.6+ KB


    4.1 Оценки пользователей

Разделим все игры по оценкам пользователей и выделим категории: 
- высокая оценка (от 8 до 10 включительно);
- средняя оценка (от 3 до 8, не включая правую границу интервала);
- низкая оценка (от 0 до 3, не включая правую границу интервала):

In [27]:
# копия датафрейма
df_actual_rating = df_actual.copy()

# категории
df_actual_rating['user_rating_category'] = pd.cut(
    df_actual_rating['user_score'],
    bins=[0, 3, 8, 10],
    labels=['низкая', 'средняя', 'высокая'],
    right=False
)

print("\nРаспределение игр по категориям оценок:")
print(df_actual_rating['user_rating_category'].value_counts())


Распределение игр по категориям оценок:
user_rating_category
средняя    9766
высокая    3082
низкая      132
Name: count, dtype: int64


    4.2 Оценки критиков

Разделим все игры по оценкам критиков и выделим категории:
- высокая оценка (от 80 до 100 включительно);
- средняя оценка (от 30 до 80, не включая правую границу интервала);
- низкая оценка (от 0 до 30, не включая правую границу интервала):

In [28]:
# категории оценок критиков
df_actual_reviews = df_actual.copy()
df_actual_reviews['critic_rating_category'] = pd.cut(
    df_actual_reviews['critic_score'],
    bins=[0, 30, 80, 100],
    labels=['низкая', 'средняя', 'высокая'],
    right=False
)
df_actual_critic_reviews = df_actual_reviews['critic_rating_category'].value_counts()
print("Распределение игр по оценкам критиков:")
print(df_actual_critic_reviews)

Распределение игр по оценкам критиков:
critic_rating_category
средняя    11098
высокая     1822
низкая        60
Name: count, dtype: int64


    4.3 Топ-7 платформ

Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период:

In [29]:
# количество игр по каждой платформе в переменной 'platform_counts' и вывод по условию в 'top_platforms'
platform_counts = df_actual['platform'].value_counts()
top_platforms = platform_counts.head(7)

print("Топ-7 платформ по количеству игр:")
print(top_platforms)

Топ-7 платформ по количеству игр:
platform
PS2     2154
DS      2146
WII     1294
PSP     1199
X360    1138
PS3     1107
GBA      826
Name: count, dtype: int64


Заключение по категоризации данных:
* [x] проведен анализ распределения игр по категориям пользовательских оценок
* [x] исследовано распределение игр по оценкам критиков
* [x] определен топ-7 платформ по количеству выпущенных игр<p>

Полученные результаты пользовательских оценок говорят о том, что средняя категория доминирует с 9766 играми <p>
Оценки критиков также в большинстве своем расположились в средней категории 11098 игр <p>
В обоих случаях (пользовательские и критические оценки) большинство игр попадает в среднюю категорию, однако высокая оценка встречается значительно реже, чем средняя

Несколько слов о платформах - лидирующая группа безусловно PS2 и DS (речь о Nintendo DS) с практически одинаковым количеством игр 2154 и 2146 соответственно, лидирующие платформы показывают практически идентичные результаты, платформы Sony и Nintendo доминируют на рынке
</div> 

---
##### Заключение

Цель исследования заключалась в анализе развития игровой индустрии начала XXI века для создания статьи, направленной на привлечение новой аудитории к игре «Мир анализа данных».

В ходе работы был проведён анализ исторических данных игровой индустрии за период с 2000 по 2013 год.

Временной анализ показал динамику роста количества игр: от 357 в 2000 году до пика в 1450 игр в 2009 году.

Оценка качества выявила, что большинство игр получили средние оценки как от пользователей (9766 игр), так и от критиков (11098 игр).

Платформенный анализ определил лидеров рынка: PlayStation 2 и Nintendo DS с более чем 2000 игр каждая.

Особое внимание было уделено работе с неполными данными:

- [x] разработана методика заполнения пропусков в оценках на основе жанра, платформы и года выпуска;
- [x] определён подход к обработке рейтинга ESRB;
- [x] проведён отбор значимых данных с удалением несущественных пропусков.

##### Результаты исследования демонстрируют значительный рост индустрии игр в рассматриваемый период, а анализ оценок показывает преобладание игр среднего качества. Подготовленные данные для команды игры «Мир анализа данных» помогут подготовить интересную статью о развитии индустрии игр в начале XXI века, опираясь на полученные инсайты и привлечь новую аудиторию!
